In [1]:

# TableName='BudgetsapActual'
# df = spark.read.option("multiline", "true").option("overwriteSchema", "true").option("header","true").json("Files/sap/"+ TableName +"/SAP-actual-data*.json")
# # # display(df)
# # df.printSchema()
# # # # df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/" + TableName)

StatementMeta(, 160c7162-d02f-476d-a405-75a6000651f8, 3, Finished, Available, Finished)

In [1]:
# Define the base path and table name
TableName = 'BudgetsapActual'
base_path = f"Files/sap/{TableName}/"
# List all files in the directory using Hadoop API
all_files = [file.getPath().getName() for file in spark.sparkContext._jvm.org.apache.hadoop.fs.FileSystem
             .get(spark._jsc.hadoopConfiguration()).listStatus(spark._jvm.org.apache.hadoop.fs.Path(base_path))
             if file.getPath().getName().startswith("SAP-actual-data") and file.getPath().getName().endswith(".json")]

# Check if any files exist
if not all_files:
    raise FileNotFoundError("No JSON files found in the specified directory.")


latest_file = max(all_files, key=lambda x: x.split('SAP-actual-data')[-1].replace('.json', ''))

# Construct the full path to the latest file
latest_file_path = base_path + latest_file

# Load the latest JSON file
df = spark.read.option("multiline", "true").option("overwriteSchema", "true").option("header", "true").json(latest_file_path)

# Print the schema of the loaded dataframe
df.printSchema()

StatementMeta(, c9c09498-d059-4509-9cf7-0ecfdfec3048, 3, Finished, Available, Finished)

root
 |-- d: struct (nullable = true)
 |    |-- Gjahr: string (nullable = true)
 |    |-- ItemSet: struct (nullable = true)
 |    |    |-- results: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- Currency: string (nullable = true)
 |    |    |    |    |-- DIC: string (nullable = true)
 |    |    |    |    |-- Gjahr: string (nullable = true)
 |    |    |    |    |-- Khoan_muc: string (nullable = true)
 |    |    |    |    |-- Model: string (nullable = true)
 |    |    |    |    |-- Month_1: string (nullable = true)
 |    |    |    |    |-- Month_10: string (nullable = true)
 |    |    |    |    |-- Month_11: string (nullable = true)
 |    |    |    |    |-- Month_12: string (nullable = true)
 |    |    |    |    |-- Month_2: string (nullable = true)
 |    |    |    |    |-- Month_3: string (nullable = true)
 |    |    |    |    |-- Month_4: string (nullable = true)
 |    |    |    |    |-- Month_5: string (nullable = true)
 |   

In [2]:

selected_columns = ['d.Gjahr', 'd.ItemSet', 'd.Rbukrs', 'd.__metadata']
new_df = df.select(*selected_columns)
new_df.printSchema()
new_df.show()

StatementMeta(, c9c09498-d059-4509-9cf7-0ecfdfec3048, 4, Finished, Available, Finished)

root
 |-- Gjahr: string (nullable = true)
 |-- ItemSet: struct (nullable = true)
 |    |-- results: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- Currency: string (nullable = true)
 |    |    |    |-- DIC: string (nullable = true)
 |    |    |    |-- Gjahr: string (nullable = true)
 |    |    |    |-- Khoan_muc: string (nullable = true)
 |    |    |    |-- Model: string (nullable = true)
 |    |    |    |-- Month_1: string (nullable = true)
 |    |    |    |-- Month_10: string (nullable = true)
 |    |    |    |-- Month_11: string (nullable = true)
 |    |    |    |-- Month_12: string (nullable = true)
 |    |    |    |-- Month_2: string (nullable = true)
 |    |    |    |-- Month_3: string (nullable = true)
 |    |    |    |-- Month_4: string (nullable = true)
 |    |    |    |-- Month_5: string (nullable = true)
 |    |    |    |-- Month_6: string (nullable = true)
 |    |    |    |-- Month_7: string (nullable = true)
 |    |    |   

In [4]:
from pyspark.sql.functions import col, explode

df_exploded = new_df.withColumn("result", explode(col("ItemSet.results")))

newest_df = df_exploded.select(
    "result.Currency", 
    "result.DIC", 
    "result.Gjahr", 
    "result.Khoan_muc", 
    "result.Model", 
    "result.Month_1", 
    "result.Month_2", 
    "result.Month_3", 
    "result.Month_4", 
    "result.Month_5", 
    "result.Month_6", 
    "result.Month_7", 
    "result.Month_8", 
    "result.Month_9", 
    "result.Month_10", 
    "result.Month_11", 
    "result.Month_12", 
    "result.Prctr", 
    "result.Rbukrs", 
    "result.Rbusa", 
    "result.Rcntr", 
    "result.Store"
)

# newest_df.show()
newest_df.printSchema()

StatementMeta(, 160c7162-d02f-476d-a405-75a6000651f8, 6, Finished, Available, Finished)

root
 |-- Currency: string (nullable = true)
 |-- DIC: string (nullable = true)
 |-- Gjahr: string (nullable = true)
 |-- Khoan_muc: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Month_1: string (nullable = true)
 |-- Month_2: string (nullable = true)
 |-- Month_3: string (nullable = true)
 |-- Month_4: string (nullable = true)
 |-- Month_5: string (nullable = true)
 |-- Month_6: string (nullable = true)
 |-- Month_7: string (nullable = true)
 |-- Month_8: string (nullable = true)
 |-- Month_9: string (nullable = true)
 |-- Month_10: string (nullable = true)
 |-- Month_11: string (nullable = true)
 |-- Month_12: string (nullable = true)
 |-- Prctr: string (nullable = true)
 |-- Rbukrs: string (nullable = true)
 |-- Rbusa: string (nullable = true)
 |-- Rcntr: string (nullable = true)
 |-- Store: string (nullable = true)



In [5]:
from pyspark.sql.functions import *
df_with_load_time = newest_df.withColumn("load_time", lit(current_timestamp()))

StatementMeta(, 160c7162-d02f-476d-a405-75a6000651f8, 7, Finished, Available, Finished)

In [6]:
newest_df = newest_df.withColumn("_insertAt", lit(current_timestamp()))

StatementMeta(, 160c7162-d02f-476d-a405-75a6000651f8, 8, Finished, Available, Finished)

In [7]:
from pyspark.sql.functions import col, concat_ws, lit, current_timestamp
newest_df = newest_df.withColumn("Id", concat_ws("_", col("DIC"), col("Gjahr"),col("Model"), col("Khoan_muc"), col("Store")))

StatementMeta(, 160c7162-d02f-476d-a405-75a6000651f8, 9, Finished, Available, Finished)

In [8]:
old_df=spark.sql("Select * from BudgetsapActual where Id is not null ")

StatementMeta(, 160c7162-d02f-476d-a405-75a6000651f8, 10, Finished, Available, Finished)

In [11]:
# from delta.tables import DeltaTable

# # Define the target table (existing Delta Table)
# delta_table = DeltaTable.forName(spark, "BudgetsapActual")

# delta_table.alias("source") \
#     .merge(newest_df.alias("target"), 
#            "target.Id = source.Id") \
#     .whenNotMatchedInsert(values={
#         "Id": "target.Id"
#     }) \
#     .execute()


StatementMeta(, 160c7162-d02f-476d-a405-75a6000651f8, 13, Finished, Available, Finished)

In [13]:
from delta.tables import DeltaTable
delta_table = DeltaTable.forName(spark, "BudgetsapActual")

delta_table.alias("old").merge(
    newest_df.alias("new"),
    "old.Id = new.Id"
).whenMatchedUpdateAll(
    condition="new._insertAt > old._insertAt"
).whenNotMatchedInsertAll().execute()


StatementMeta(, 160c7162-d02f-476d-a405-75a6000651f8, 15, Finished, Available, Finished)

In [2]:
%%sql
SELECT * FROM BudgetsapActual WHERE Gjahr=2025

StatementMeta(, 520309a5-7aea-4904-83dc-1b2378902b4f, 3, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 24 fields>

In [19]:
%%sql
SELECT Id, COUNT(*) 
FROM BudgetsapActual
GROUP BY Id
HAVING COUNT(*) > 1

StatementMeta(, 160c7162-d02f-476d-a405-75a6000651f8, 21, Submitted, Running, Running)

<Spark SQL result set with 0 rows and 2 fields>